Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy, scipy.stats t_ és _norm_, valamint a _scipy.optimize_ _minimize_ osztályai 

In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

In [ ]:
datalib = Path.cwd().parent.joinpath('data')

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

In [4]:
import numpy as np
x = [0.1, 0.1, 0.1, 0.1, 0.1]
type(x)
np.array(x)

array([0.1, 0.1, 0.1, 0.1, 0.1])

### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot. <u>Amennyiben működött, önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályt is.</u>

7., Készíts egy új __LinearRegressionML__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami ML elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között, amely az első változó legyen. __A feladatot numerikus optimalizálással old meg, analítikus megoldásra nem ját pont__

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj, hogy  alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

#### Kiegészítés:
- Mivel a célparaméterek kicsik, a minimalizálás kiinduló értékeit érdemes egységesen, minden paraméter számára __0.1__-re állítani.
- Javasolt a __L-BFGS-B__ optimalizáló alkalmazása
- Ne felejtsd el, hogy az MLE __variancia becslése torzított__. A kapott eredményeket ennek megfelelően korrigáld.

In [12]:
def loglikelihood(params, X, Y):
    beta = params[:-1]
    sigma_squared = params[-1]
    n = len(Y)
    residuals = Y - np.dot(X, beta)
    log_likelihood = -0.5 * n * np.log(2 * np.pi * sigma_squared) - 0.5 * np.sum(residuals**2) / sigma_squared
    return -log_likelihood

In [9]:
import pandas as pd
import numpy as np
amazon_df = pd.read_parquet('/Users/kissr/Downloads/Programming/Python/ECOPY_23241/data/weekly6/toclean.parquet', engine='fastparquet')
amazon_df.insert(0, 'alpha', 1)
y = amazon_df['ex_ret_1']
x = amazon_df[['alpha', 'Mkt-RF', 'SMB', 'HML']]

In [16]:
import scipy
params = [0.1, 0.1, 0.1, 0.1, 0.1]
loglikelihood(params, x, y)
res = scipy.optimize.minimize(loglikelihood(params, x, y), params)
res.x

TypeError: 'numpy.float64' object is not callable